Import Tensorflow and open a session.

In [2]:
%run 'Ising.py'

### Determinant
Calculate matrix determinant

In [4]:
W = tf.constant([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.matrix_determinant(W)
tf.Session().run(f)

9.0

Train by minimizing the ln(det(W)). TF knows how to do the gradient.

In [5]:
W = tf.Variable([[0.,1.,1.,1.],
                 [-1.,0.,1.,-1.],
                 [-1.,-1.,0.,1.],
                 [-1.,1.,-1.,0.]])
f = tf.log(tf.matrix_determinant(W))
init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(f)
sess = tf.Session()
sess.run(init)
for i in range(5):
    sess.run(train)
    print(sess.run(f))

2.06162
1.91632
1.75986
1.5904
1.4056


### Sparse Tensor
Sparse tensor must be converted to dense when passed to the determinant op.

In [337]:
W = tf.SparseTensor(indices=[[0, 0], [1, 1]], values=[1., 2.], dense_shape=[2, 2])
W_dense = tf.sparse_tensor_to_dense(W)
f = tf.log(tf.matrix_determinant(W_dense))
sess.run(f)

0.69314718

## Entanglement
### Entanglement Regions

In [43]:
list(Region({1,2},4).equivalences())

[{2, 3}, {0, 1}, {1, 2}, {0, 3}]

In [44]:
Region({1,2},4).config()

array([ 1.000, -1.000, -1.000,  1.000])

In [42]:
list(Regions(8).consecutives())

[set(),
 {0},
 {0, 1},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7}]

In [41]:
list(Regions(8).representatives())

[set(),
 {0},
 {0, 1},
 {0, 2},
 {0, 3},
 {0, 4},
 {0, 1, 2},
 {0, 1, 3},
 {0, 1, 4},
 {0, 2, 4},
 {0, 2, 5},
 {0, 1, 2, 3},
 {0, 1, 2, 4},
 {0, 1, 2, 5},
 {0, 1, 3, 5},
 {0, 1, 3, 6},
 {0, 1, 4, 5},
 {0, 2, 4, 6}]

### Entanglement Entropy
Entanglement entropy of free fermion.
$$H=\sum_{i}(1+(-)^i m)\mathrm{i}\chi_{i}\chi_{i+1}$$

In [745]:
system = FreeFermion(mass=1.e-10, size=8)
system.S([0,1,2,3,4])

0.84523244468672853

### Square Lattice Adjacency Matices

In [143]:
%run 'Ising.py'

In [709]:
%run 'Ising.py'
machine = EFL(FreeFermion(mass=1.e-10, size=256), CylindricalModel(Lx = 8, Lz = 2))
training_set = machine.training_set('representatives')
writer = machine.writer()
writer.add_graph(sess.graph)
sess.run(machine.model.initialize)

In [712]:
for i in range(2000):
    machine.step += 1
    sess.run(machine.model.train, training_set)
    writer.add_summary(sess.run(machine.model.summarize, training_set), machine.step)
sess.run([machine.model.MSE, machine.model.L2, machine.model.cost], training_set)

[7.0578566877442751e-06, 0.0, 7.0578566877442751e-06]

In [559]:
for i in range(20):
    machine.step += 1
    sess.run(machine.model.train, training_set)
    if machine.step%5 == 0:
        print(sess.run([machine.model.MSE, machine.model.cost], training_set))
    #writer.add_summary(sess.run(machine.model.summary, training_set), step)

[0.00036799669779277169, 0.00036799669779277169]
[0.000367434595445579, 0.000367434595445579]
[0.00036680721552214551, 0.00036680721552214551]
[0.00036438687074209932, 0.00036438687074209932]
[0.00033017774864124445, 0.00033017774864124445]
[8.2275243147926714e-05, 8.2275243147926714e-05]
[0.00029979805222174558, 0.00029979805222174558]
[2.185024256956648e-05, 2.185024256956648e-05]
[1.1591512976705274e-05, 1.1591512976705274e-05]
[8.6165197012395949e-06, 8.6165197012395949e-06]
[7.6213167368117497e-06, 7.6213167368117497e-06]
[8.9957979431971903e-06, 8.9957979431971903e-06]
[7.5626068300154472e-06, 7.5626068300154472e-06]
[7.2163256752003698e-06, 7.2163256752003698e-06]
[2.0883367511462706e-05, 2.0883367511462706e-05]
[7.3570019367415802e-06, 7.3570019367415802e-06]
[7.127907164592016e-06, 7.127907164592016e-06]
[9.3441658623608355e-06, 9.3441658623608355e-06]
[7.1884480382420105e-06, 7.1884480382420105e-06]
[8.9219668059241842e-06, 8.9219668059241842e-06]
[7.2667523359608649e-06, 7.2

In [610]:
sess.run([machine.model.Jx, machine.model.Jz, machine.model.h], training_set)

[array([ 0.145]), array([ 0.329,  0.886]), 1.9557621280904396]

In [611]:
training_set[machine.model.Ssys]-sess.run(machine.model.Smdl, training_set)

array([ 0.010,  0.003,  0.004,  0.006,  0.011,  0.001, -0.005, -0.002,
       -0.002, -0.006, -0.001, -0.003, -0.006, -0.010, -0.013, -0.008,
       -0.006])

In [697]:
with tfgraph.as_default():
    print(sess.run(machine.model.optimizer.compute_gradients(machine.model.cost,[machine.model.Jx]), training_set))

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("confs:0", shape=(?, 8), dtype=float64) is not an element of this graph.

In [585]:
machine.model.optimizer.get_slot(machine.model.Jz,'m')

<tf.Variable 'J/Jz/Adam:0' shape=(2,) dtype=float64_ref>

In [527]:
training_set[machine.model.Ssys]-sess.run(machine.model.Smdl, training_set)

array([ 0.013,  0.004,  0.013,  0.010,  0.014,  0.006,  0.002, -0.002,
        0.011,  0.002,  0.005,  0.004, -0.005, -0.002, -0.004, -0.013,
        0.014])

In [53]:
sess.run(machine.model.MSL, training_set)

nan

In [686]:
machine.model.Jx.name

'J/Jx:0'

In [13]:
sess.run(a)

0.5

In [12]:
sess.run(a.assign(0.5))

0.5

In [98]:
'./log/Lx{0}|{3}_Lz{1}_mass{2:.2f}_{4:.3}/'.format(8,4,1.e-10,128,'representatives')

'./log/Lx8|128_Lz4_mass0.00_rep/'

In [113]:
para = {'Jx0': 0.2, 'Jz0': 0.2, 'h': 1., 'fr': 0.0001}
''.join(['{0}:{1:.3g}_'.format(name, value) for (name, value) in para.items()])

'fr:0.0001_Jx0:0.2_Jz0:0.2_h:1_'

In [591]:
for i in []:
    print(i)